# Import libraries

In [99]:
import os
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-11'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '\\bin;' + os.environ['PATH']
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

# Create Spark Session

In [100]:
spark = SparkSession.builder.appName("Churn Prediction").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

# Create dataframe

In [ ]:
df_path = r'data\WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = spark.read.csv(df_path,header=True)
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [ ]:
df.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [ ]:
# Change certain types to numeric
to_numeric = ['tenure', 'MonthlyCharges', 'TotalCharges']

for col in to_numeric:
    df = df.withColumn(col, F.col(col).cast(FloatType()))

df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: float (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: float (nullable = true)
 |-- TotalCharges: float (nullable = true)
 |-- Churn: string (nullable = true)



# Data Cleaning

In [ ]:
# Delete ID as it is unnecessary
df = df.drop('CustomerID')

for i in range(3):
    # Check for null values and outliers
    print('\nChecking for null_values')
    for col in df.columns:
        null_count = df.filter(F.col(col).isNull()).count()
        if null_count>0:
            print(f"Column '{col}' has {null_count} null values")
            first_value_type = type(df.select(col).first()[0])

            if first_value_type==str:
                most_frequent_value = df.groupby(col).count().orderBy('count',ascending=False).first()[0]
                df = df.fillna({col: most_frequent_value})
            else:
                avg_value = df.agg(F.avg('TotalCharges')).collect()[0][0]
                df = df.fillna({col: avg_value})

    print('\nChecking for outliers')
    for col in df.columns:
        first_value_type = type(df.select(col).first()[0])

        if first_value_type!=str:    
            quantiles = df.approxQuantile(col, [0.25,0.75],0.01)
            Q1 = quantiles[0]
            Q3 = quantiles[1]
            IQR = Q3-Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = df.filter((F.col(col) < lower_bound) | (F.col(col) > upper_bound))
            if outliers.count()>0:
                print(f"Column '{col}' has {outliers.count()} outliers")
                df = df.filter((F.col(col) >= lower_bound) & (F.col(col) <= upper_bound))


Checking for null_values
Column 'TotalCharges' has 11 null values

Checking for outliers


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\user\PycharmProjects\CustomerChurnPrediction\.conda\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\user\PycharmProjects\CustomerChurnPrediction\.conda\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\user\PycharmProjects\CustomerChurnPrediction\.conda\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\user\PycharmProjects\CustomerChurnPrediction\.conda\lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: E

Py4JError: An error occurred while calling o6575.approxQuantile